# **Loading the data:**

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Project 2/SBAnational.csv')

# **Cleaning:**

In [ ]:
df.head()

In [ ]:
df = df.rename(columns={'LoanNr_ChkDgt':'ID'})

In [ ]:
df = df.drop(['BalanceGross'], axis=1)

In [ ]:
df['DisbursementGross'] = df['DisbursementGross'].str.replace('$','')

In [ ]:
df['DisbursementGross'] = df['DisbursementGross'].str.replace(',','')

In [ ]:
df['DisbursementGross'] = df['DisbursementGross'].str.strip()

In [ ]:
df['DisbursementGross'] = pd.to_numeric(df['DisbursementGross'])

In [ ]:
df['GrAppv'] = df['GrAppv'].str.replace('$','')

In [ ]:
df['GrAppv'] = df['GrAppv'].str.replace(',','')

In [ ]:
df['GrAppv'] = df['GrAppv'].str.strip()

In [ ]:
df['GrAppv'] = pd.to_numeric(df['GrAppv'])

In [ ]:
df['SBA_Appv'] = df['SBA_Appv'].str.replace('$','')

In [ ]:
df['SBA_Appv'] = df['SBA_Appv'].str.replace(',','')

In [ ]:
df['SBA_Appv'] = df['SBA_Appv'].str.strip()

In [ ]:
df['SBA_Appv'] = pd.to_numeric(df['SBA_Appv'])

In [ ]:
df['MIS_Status'] = df['MIS_Status'].str.strip()

In [ ]:
df.isnull().sum() / len(df) * 100

In [ ]:
df.dtypes

In [ ]:
df.shape

In [ ]:
df.corr()

In [ ]:
df.columns

# **Urban vs Rural**

In [ ]:
# what is the average term in rural and urban areas
# 1 = Urban
# 2 = Rural
# 0 = Undefined
URterm = df.groupby('UrbanRural').agg({'Term': [np.mean]})
URterm

***Urban areas have a shorter loan term than Rural areas***

In [ ]:
# SBA_appv is SBAs guaranteed loan approved amount
# Grappv is Gross amount of loan approved by bank
UrbanRuraldf = df.groupby('UrbanRural').agg({'GrAppv': [np.mean], 'SBA_Appv': [np.mean], 'DisbursementGross': [np.mean]})
UrbanRuraldf


In [ ]:
UrbanRuraldf.reset_index()

In [ ]:
UrbanRuraldf.columns = ['Average Bank Loan', 'Average SBA Guaranteed Loan Amount', 'Disburstment Amount']

In [ ]:
UrbanRuraldf = UrbanRuraldf.reset_index()

In [ ]:
UrbanRuraldf

In [ ]:
UrbanRuraldf['Difference in Amount'] = UrbanRuraldf['Average Bank Loan'] - UrbanRuraldf['Average SBA Guaranteed Loan Amount']

In [ ]:
UrbanRuraldf

In [ ]:
Urbanloans = UrbanRuraldf[UrbanRuraldf['UrbanRural'] == 1]
Urbanloans

In [ ]:
Ruralloans = UrbanRuraldf[UrbanRuraldf['UrbanRural'] == 2]

In [ ]:
plt.figure(figsize=(12,8))
plt.bar(Urbanloans['UrbanRural'], Urbanloans['Average Bank Loan'], color ='green',width = 0.4, label='Avg. Bank Loan')
plt.bar(Urbanloans['UrbanRural'], Urbanloans['Average SBA Guaranteed Loan Amount'], color ='red',width = 0.4, label='Avg. SBA Loan')
plt.bar(Ruralloans['UrbanRural'], Ruralloans['Average Bank Loan'], color ='green',width = 0.4)
plt.bar(Ruralloans['UrbanRural'], Ruralloans['Average SBA Guaranteed Loan Amount'], color ='red', width = 0.4)
plt.rc('font', size=20)
plt.ylabel('Loan Amount')
plt.rc('font', size=20)
plt.title('Bank vs SBA Loans in Urban and Rural Areas')
plt.xticks(UrbanRuraldf['UrbanRural'], labels=['','Urban','Rural'])
plt.legend(loc='lower left')
plt.show()

# **Term:**

In [ ]:
df.MIS_Status.replace(['P I F', 'CHGOFF'], [0,1], inplace = True)

In [ ]:
termdefault = df.groupby('Term').agg({'MIS_Status': ['sum', 'count'], 'DisbursementGross': [np.mean, 'sum']})


In [ ]:
termdefault.reset_index()

In [ ]:
termdefault.columns = ['Defaulted' , 'Total', 'AVG Loan Amount', 'Total Loan Amount']

In [ ]:
termdefault = termdefault.reset_index()

In [ ]:
termdefault['percentage of defaults'] = termdefault['Defaulted'] / termdefault['Total'] *100

In [ ]:
plt.figure(figsize=(12,8))
plt.scatter(termdefault['Term'], termdefault['AVG Loan Amount'])
plt.title('Loan Amount by Term')
plt.xlabel('Term')
plt.ylabel('Loan Amount')

In [ ]:
plt.figure(figsize=(12,8))
plt.bar(termdefault['Term'], termdefault['percentage of defaults'], color='red')
plt.title('Defaulted in Term')
plt.xlabel('Term')
plt.ylabel('Percentage of Defaults')

# **Revolving line of credit (most of the values are missing)**

In [ ]:
df.columns

In [ ]:
df['RevLineCr'] = df['RevLineCr'].map(dict(Y=1, N=0))

In [ ]:
RLC = df.groupby('UrbanRural').agg({'RevLineCr': [np.sum, 'count']})
RLC

In [ ]:
RLC.reset_index()

In [ ]:
RLC.columns = ['Total RLC', "# of people"]

In [ ]:
RLC = RLC.reset_index()

In [ ]:
RLC['% of people w RLC'] = RLC['Total RLC'] / RLC['# of people'] * 100

# **Low Doc**

In [ ]:
df['LowDoc'] = df['LowDoc'].map(dict(Y=1, N=0))

In [ ]:
LowDocUR = df.groupby('UrbanRural').agg({'LowDoc': [np.mean, np.sum, 'count'], 'MIS_Status': ['sum']})
LowDocUR

In [ ]:
LowDocUR.reset_index()
LowDocUR

In [ ]:
LowDocUR.columns = ['Average','Approved', 'Total', 'Defaulted']

In [ ]:
LowDocUR = LowDocUR.reset_index()
LowDocUR

In [ ]:
LowDocUR['% Approved'] = LowDocUR['Approved'] / LowDocUR['Total'] * 100

In [ ]:
LowDocUR

In [ ]:
lowdoc = df.groupby('LowDoc').agg({'MIS_Status': ['count', 'sum']})

In [ ]:
lowdoc.reset_index()

In [ ]:
lowdoc.columns = ['Total', 'Defaulted']

In [ ]:
lowdoc = lowdoc.reset_index()

In [ ]:
lowdoc['Percentage of Defaults'] = lowdoc['Defaulted'] / lowdoc['Total'] *100

In [ ]:
lowdoc

In [ ]:
plt.figure(figsize=(12,8))
plt.bar(lowdoc['LowDoc'], lowdoc['Percentage of Defaults'], color ='red',width = 0.4)
plt.rc('font', size=20)
plt.ylabel('Percentage of Defaults')
plt.rc('font', size=20)
plt.title('Defaults Within The LowDoc Program')
plt.xticks(lowdoc['LowDoc'], labels=['No','Yes'])
plt.show()

# **MIS Status**

In [ ]:
df.MIS_Status.replace(['P I F', 'CHGOFF'], [0,1], inplace = True)

In [ ]:
statusUR = df.groupby('UrbanRural').agg({'MIS_Status': [np.sum, 'count']})
statusUR

In [ ]:
statusUR.reset_index()

In [ ]:
statusUR.columns = ['Defaulted', 'Total Loans']

In [ ]:
statusUR = statusUR.reset_index()

In [ ]:
statusUR

In [ ]:
statusUR['% of Defaulted'] = statusUR['Defaulted'] / statusUR['Total Loans'] * 100

In [ ]:
def make_scatterplot(df, x, y, figsize=(12, 8)):
    df.plot(x=x, 
            y=y, 
            kind='bar', 
            figsize=figsize,
            title=f'{x} vs {y}')

In [ ]:
make_scatterplot(statusUR, x='UrbanRural', y='% of Defaulted')

In [ ]:
statusUR.columns

In [ ]:
statusUR

In [ ]:
statestatus = df.groupby('State').agg({'MIS_Status': ['sum', 'count'], 'GrAppv': [np.mean], 'SBA_Appv': [np.mean]})

In [ ]:
statestatus.reset_index()

In [ ]:
statestatus.columns = ['Defaulted', 'Total', 'Bank Loan', 'SBA Loan']

In [ ]:
statestatus = statestatus.reset_index()

In [ ]:
statestatus['Percentage of Defaulted'] = statestatus['Defaulted'] / statestatus['Total'] * 100

In [ ]:
statestatus.columns 

In [ ]:
plt.figure(figsize=(15,10))
plt.bar(statestatus['State'], statestatus['Total'], color = ['r','r','r','r','b','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','b','r','r','r','r','r','r','r','r','b','r','r','r','r','r','r'])

plt.rc('font', size=20)
plt.xlabel('State')
plt.ylabel('Total vs Default')
plt.rc('font', size=20)
plt.title('Count of Loans in Each State')
plt.xticks(rotation = 90)
plt.legend(loc='top right')
plt.savefig('Stateloancounts.png')
plt.show()

In [ ]:
plt.figure(figsize=(15,10))
plt.bar(statestatus['State'], statestatus['Total'], color = ['r','r','r','r','b','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','b','r','r','r','r','r','r','r','r','b','r','r','r','r','r','r'])

plt.rc('font', size=20)
plt.xlabel('State')
plt.ylabel('Count')
plt.rc('font', size=20)
plt.title('Count of Loans in Each State')
plt.xticks(rotation = 90)
plt.savefig('Stateloancountss.png')
plt.show()

In [ ]:
df_sorted_desc= statestatus.sort_values('Percentage of Defaulted',ascending=False)


In [ ]:
plt.figure(figsize=(15,10))
plt.bar(statestatus['State'], statestatus['Percentage of Defaulted'],data=df_sorted_desc, color=['r','r','r','r','r','r','r','b','r','b','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r','r'])
plt.rc('font', size=20)
plt.xlabel('State')
plt.ylabel('Percentage of Defaults')
plt.rc('font', size=20)
plt.title('Defaults in Each State')
plt.xticks(rotation = 90)
plt.savefig('newStatedefaults.png')
plt.show()

# **NACIS**




In [ ]:
sectordf = pd.read_csv('/content/drive/MyDrive/NAICS Sector - Sheet1 - NAICS Sector - Sheet1 (1).csv')

In [ ]:
df['Sectornum'] = df['NAICS'].astype(str).str[:2].astype(int)

In [ ]:
sectordf.head()

In [ ]:
df.head()

In [ ]:
NAICSsector = pd.merge(sectordf, df, left_on='Sector', right_on='Sectornum', how='right')

In [ ]:
NAICSsector.head()

In [ ]:
len(df)

In [ ]:
len(NAICSsector)

In [ ]:
NAICSsector.to_csv('data_with_sectors.csv', index=False)

In [ ]:
NAICSDESSBA = NAICSsector.groupby(['Description', 'UrbanRural']).agg({'SBA_Appv': [np.mean, 'count', 'min', 'max'], 'MIS_Status': ['sum']})

In [ ]:
NAICSDESSBA.reset_index()

In [ ]:
NAICSDESSBA.columns= ['Average', 'Count of Businesses', 'Minimum Loan', 'Maximum Loan', 'Amount Defaulted']

In [ ]:
NAICSDESSBA = NAICSDESSBA.reset_index()

In [ ]:
NAICSDESSBA['Percentage of Defaulted'] = NAICSDESSBA['Amount Defaulted'] / NAICSDESSBA['Count of Businesses'] * 100

In [ ]:
SectorUrban = NAICSDESSBA[NAICSDESSBA['UrbanRural'] == 1]

In [ ]:
SectorRural = NAICSDESSBA[NAICSDESSBA['UrbanRural'] == 2]

In [ ]:
plt.figure(figsize=(12,8))
plt.barh(SectorUrban['Description'], SectorUrban['Percentage of Defaulted'], label='Urban')
plt.barh(SectorRural['Description'], SectorRural['Percentage of Defaulted'], label='Rural')
plt.rc('font', size=20)
plt.xlabel('Percentage That Have Defaulted')
plt.rc('font', size=20)
plt.legend(loc='top right')
plt.title('Urban vs Rural Businesses That Have Defaulted')
plt.show()